In [1]:
import numpy
from netgen.csg import *
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.webgui import Draw

In [2]:
geo = SplineGeometry()
# Define your geometry here (e.g., boundaries, domain, electrodes)
p0 = geo.AppendPoint(0,0)
p1 = geo.AppendPoint(1,0)
p2 = geo.AppendPoint(1,1)
p3 = geo.AppendPoint(0,1)

line0 = geo.Append(['line', p0, p1], leftdomain=1, rightdomain=0)
line1 = geo.Append(['line', p1, p2], leftdomain=1, rightdomain=0)
line2 = geo.Append(['line', p2, p3], leftdomain=1, rightdomain=0)
line3 = geo.Append(['line', p3, p0], leftdomain=1, rightdomain=0)

mesh = Mesh(geo.GenerateMesh(maxh=0.1))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [70]:
V = H1(mesh, order=2, dirichlet="line0|line1")  # For the electric potential
W = VectorH1(mesh, order=2)  # For the electric field

In [71]:
u = V.TrialFunction()
v = V.TestFunction()
gfu = GridFunction(V)

a = BilinearForm(V)
a += grad(u) * grad(v)*dx
a.Assemble()

# Sources
source_location = [0.5,0.5]
source_strength = 1e-15
source_expr = source_strength * (1 / sqrt((x - source_location[0])**2 + (y - source_location[1])**2))

f = LinearForm(V)
f += source_expr*v*dx
f.Assemble()

gfu.vec.data = a.mat.Inverse(freedofs=V.FreeDofs()) * f.vec


In [72]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [60]:
grad_u = GridFunction(W)  # Create a GridFunction for the electric field
grad_u.Set(grad(gfu))

# Now grad_u contains the electric field